In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Import CIFAR-100 and Resize

In [3]:
resize = transforms.Compose([transforms.Resize(64), transforms.ToTensor()])

In [4]:
train_set = datasets.CIFAR100(root="data", train=True, download=True, transform=resize)

Files already downloaded and verified


In [5]:
test_set = datasets.CIFAR100(root="data", train=False, download=True, transform=resize)

Files already downloaded and verified


In [6]:
train_set[0][0].shape

torch.Size([3, 64, 64])

In [7]:
test_set[0][0].shape

torch.Size([3, 64, 64])

# VGG16

In [8]:
import torch.nn as nn
import torchvision.models as models


model = models.vgg16(pretrained=True) #130million+ parameters


#Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False


#Add on classifier
n_classes=100
n_inputs=4096
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, n_classes))

In [9]:
model=model.to(device)

In [10]:
device

device(type='cuda')

In [11]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Training

In [12]:
train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=True)

In [13]:
criterion=torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [14]:
n_epochs = 50
train_loss = []
for epoch in range(n_epochs):
  for data, targets in train_dataloader:
    data=data.to(device)
    targets=targets.to(device)
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    train_loss.append(loss)
    #Reset the gradients
    optimizer.zero_grad()
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()

  #Evaluation
  model.eval()

  #Train Evaluation
  train_acc = 0
  train_acc_l = []
  for data, targets in train_dataloader:
    data = data.to(device)
    targets = targets.to(device)
    ps = model(data)
    #Get index of class label
    _,preds = torch.max(ps,1)
    #Get accuracy
    train_acc += torch.sum(preds == targets)
  train_acc_l.append(train_acc/500)

  #Test Evaluation
  test_acc = 0
  test_acc_l = []
  for data, targets in test_dataloader:
    data = data.to(device)
    targets = targets.to(device)
    ps = model(data)
    #Get index of class label
    _,preds = torch.max(ps,1)
    #Get accuracy
    test_acc += torch.sum(preds == targets)
  test_acc_l.append(test_acc/500)

  print(f'Epoch: {epoch+1}\t Loss: {loss:.4f}\t Train_Acc: {train_acc/500:.4f}\t Val_Acc: {test_acc/100:.4f}')

  model.train()

Epoch: 1	 Loss: 2.1265	 Train_Acc: 42.3720	 Val_Acc: 36.0700
Epoch: 2	 Loss: 2.9099	 Train_Acc: 46.9540	 Val_Acc: 37.5300
Epoch: 3	 Loss: 2.8372	 Train_Acc: 50.4140	 Val_Acc: 38.5500
Epoch: 4	 Loss: 3.7438	 Train_Acc: 51.1960	 Val_Acc: 37.7500
Epoch: 5	 Loss: 3.1873	 Train_Acc: 52.8200	 Val_Acc: 37.9700
Epoch: 6	 Loss: 3.9083	 Train_Acc: 54.2900	 Val_Acc: 38.6300
Epoch: 7	 Loss: 2.4070	 Train_Acc: 54.2340	 Val_Acc: 38.1100
Epoch: 8	 Loss: 2.3899	 Train_Acc: 56.5140	 Val_Acc: 39.2900
Epoch: 9	 Loss: 2.9042	 Train_Acc: 56.7400	 Val_Acc: 39.4000
Epoch: 10	 Loss: 1.7386	 Train_Acc: 57.1180	 Val_Acc: 38.8800
Epoch: 11	 Loss: 3.6280	 Train_Acc: 57.0000	 Val_Acc: 39.3000
Epoch: 12	 Loss: 2.3169	 Train_Acc: 57.6960	 Val_Acc: 39.2300
Epoch: 13	 Loss: 3.2330	 Train_Acc: 58.8460	 Val_Acc: 39.3300
Epoch: 14	 Loss: 2.7955	 Train_Acc: 58.5120	 Val_Acc: 39.3600
Epoch: 15	 Loss: 3.5749	 Train_Acc: 58.7320	 Val_Acc: 39.1100
Epoch: 16	 Loss: 2.8415	 Train_Acc: 58.8320	 Val_Acc: 39.1800
Epoch: 17	 Loss: 

# Testing

In [15]:
'''
model.eval()
test_acc=0
for data, targets in test_dataloader:
  data=data.to(device)
  targets=targets.to(device)
  log_ps = model(data)
  # Convert to probabilities
  ps = torch.exp(log_ps)
  #Get index of class label
  _,preds=torch.max(ps,1)
  #Get accuracy
  test_acc += torch.sum(preds == targets)

test_acc/len(test_set)
'''

'\nmodel.eval()\ntest_acc=0\nfor data, targets in test_dataloader:\n  data=data.to(device)\n  targets=targets.to(device)\n  log_ps = model(data)\n  # Convert to probabilities\n  ps = torch.exp(log_ps)\n  #Get index of class label\n  _,preds=torch.max(ps,1)\n  #Get accuracy\n  test_acc += torch.sum(preds == targets)\n\ntest_acc/len(test_set)\n'